In [57]:
import requests
import re
import urllib.parse
from pyquery import PyQuery as pq

In [69]:
session_requests = requests.session()
auth_token = None
    
login_url = 'https://danbooru.donmai.us/session/new'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
result = session_requests.get(login_url, headers = headers)

login_page = pq(result.text)
authenticity_token = urllib.parse.quote(login_page('input').attr('value'),safe='')
#print(authenticity_token)

login_data = {
    'authenticity_token': authenticity_token,
    'url': '',
    'name': 'komeiji_nori',
    'password': ,
    'remember': '1',   
    'commit': 'Submit'
}

resp = session_requests.post(url = 'https://danbooru.donmai.us/session', data = login_data, headers = headers)

In [70]:
new_url = 'https://danbooru.donmai.us/counts/posts?tags=shimakaze_(kantai_collection)+rating:safe+parent:None+amatsukaze_(kantai_collection)'
page_data = session_requests.get(new_url)
page_query = pq(page_data.text)
for tag_div in page_query('div'):
    div_query = pq(tag_div)
    div_id = div_query.attr('id')
    if div_id == 'a-posts':
        count = div_query.text()
        count = re.findall(r'\d+', count)[0]
        print(count)

999
